In [ ]:
import geopandas as gpd
import xarray_leaflet
import matplotlib.pyplot as plt
import numpy as np
import scipy.ndimage
from functools import partial
from ipyleaflet import LocalTileLayer, Map, basemaps
from geocube.rasterize import rasterize_image
from rasterio.enums import MergeAlg
from shapely.geometry import Point

In [ ]:
def transform3(array, *args, **kwargs):
    radius = 10
    circle = np.zeros((2*radius+1, 2*radius+1))
    y, x = np.ogrid[-radius:radius+1,-radius:radius+1]
    index = x**2 + y**2 <= radius**2
    circle[index] = 1
    array = np.nan_to_num(array, nan=-1)
    array = scipy.ndimage.maximum_filter(array, footprint=circle)
    array[array < 0] = np.nan
    return array

In [ ]:
df = gpd.GeoDataFrame(geometry=[
    Point(-50.31188964843751, -0.5273363048115043),
    Point(-50.52062988281251, -0.5548009001919478),
]).set_crs("epsg:4326")
df["value"] = [0, 1]

In [ ]:
m = Map(center=(-0.5273363048115043, -50.31188964843751), zoom=9, basemap=basemaps.CartoDB.DarkMatter, interpolation='nearest')
m

In [ ]:
rasterize_function = partial(rasterize_image, merge_alg=MergeAlg.replace, all_touched=False)
layer = partial(LocalTileLayer, max_zoom=20)
l = df.leaflet.plot(
    m,
    measurement="value",
    transform3=transform3,
    layer=layer,
    dynamic=False,
    rasterize_function=rasterize_function,
    colormap=plt.cm.viridis
)